In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
!pip install pillow==4.1.1
%reload_ext autoreload
%autoreload

In [0]:
import torch.utils.data
from torchvision import datasets, models, transforms
import os
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torchvision.transforms.functional as TF

In [0]:
## Random blur and random brightness adjustment to be added.
## About normalization we need to check in which values we normalize -> later
data_transforms = transforms.Compose([
        transforms.Resize((640, 512)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])

In [0]:
def generate_out_map(img_name, features, objects_to_look = ['ball'], ratio = 4):
        # ratio is the input-output ratio
        f = features[(features['image_file']==img_name) & (features.label.isin(objects_to_look))]
        out_map = torch.zeros((160, 128))

        for i in range(len(f)):
            temp = f.iloc[i]
            x_object = (temp['xmin'] + temp['xmax'])/(ratio*2) # /2 because you the middle, /4 because of the ratio with original img
            y_object = (temp['ymin'] + temp['ymax'])/(ratio*2) # /2 because you the middle, /4 because of the ratio with original img
            current_value = out_map[x_object.astype(int), y_object.astype(int)]
            if current_value == 0 or current_value < 1:
                out_map[x_object.astype(int), y_object.astype(int)] = 1

        # TODO: how to spread this one as a gaussian from the center, need discussion
        return out_map 


In [0]:
class SoccerBallDetection(Dataset):
    """Soccer Ball dataset."""

    def __init__(self, csv_file, root_dir, transform=None, objects_to_look = ['ball']):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.features = pd.read_csv(csv_file,sep=';' )
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        # Loading image
        img_name = self.features.iloc[idx, 0]
        img_path = os.path.join(self.root_dir, img_name)
        image = io.imread(img_path)
        image = torch.from_numpy(image)
        
        if self.transform:
            image = TF.to_pil_image(image)
            image = self.transform(image)
        # Extracting features
        sample = {'image': image, 'features': generate_out_map(img_name, self.features)}

        

        return sample

In [0]:
# upload files if google colab !mkdir data/train
dataset = SoccerBallDetection('data/data_10.csv', 'data/train', transform=data_transforms)

In [0]:
# import numpy as np
# import scipy.stats as st

# def gkern(kernlen=5, nsig=3):
#     """Returns a 2D Gaussian kernel array."""

#     interval = (2*nsig+1.)/(kernlen)
#     x = np.linspace(-nsig-interval/2., nsig+interval/2., kernlen+1)
#     kern1d = np.diff(st.norm.cdf(x))
#     kernel_raw = np.sqrt(np.outer(kern1d, kern1d))
#     kernel = kernel_raw/kernel_raw.sum()
#     return kernel
# gkern(3, 4)

In [0]:
dataloader = DataLoader(dataset, batch_size=4,
                        shuffle=True, num_workers=2)

In [58]:
# Just testing 
for i_batch, sample_batched in enumerate(dataloader):
    print(i_batch, sample_batched['image'].size(),
          sample_batched['features'].size())
    if(i_batch > 1):
        break

0 torch.Size([4, 3, 640, 512]) torch.Size([4, 160, 128])
1 torch.Size([4, 3, 640, 512]) torch.Size([4, 160, 128])
2 torch.Size([4, 3, 640, 512]) torch.Size([4, 160, 128])
